# Setup

In [1]:
from __future__ import print_function, division, unicode_literals

# Introduction

In [2]:
def f(x, y):
    return x*x*y + y + 2

In [3]:
# first order derivatives (also called Jacobians)
def df(x, y):
    return 2*x*y, x*x + 1

In [4]:
df(3, 4)

(24, 10)

In [5]:
# second order derivatives (also call Hessians)
def d2f(x, y):
    return [2*y, 2*x], [2*x, 0]

In [6]:
d2f(3, 4)

([8, 6], [6, 0])

# Numeric differentiation

In [7]:
def gradients(func, vars_list, eps=0.0001):
    partial_derivatives = []
    base_func_eval = func(*vars_list)
    for idx in range(len(vars_list)):
        tweaked_vars = vars_list[:]
        tweaked_vars[idx] += eps
        tweaked_func_eval = func(*tweaked_vars)
        derivative = (tweaked_func_eval - base_func_eval) / eps
        partial_derivatives.append(derivative)
    return partial_derivatives

In [8]:
def df(x, y):
    return gradients(f, [x, y])

In [9]:
df(3, 4)

[24.000400000048216, 10.000000000047748]

In [10]:
def dfdx(x, y):
    return gradients(f, [x, y])[0]

def dfdy(x, y):
    return gradients(f, [x, y])[1]

dfdx(3, 4), dfdy(3, 4)

(24.000400000048216, 10.000000000047748)

In [11]:
def d2f(x, y):
    return gradients(dfdx, [x, y]), gradients(dfdy, [x, y])

In [12]:
d2f(3, 4)

([7.999999951380232, 6.000099261882497],
 [6.000099261882497, -1.4210854715202004e-06])

# Implementing a Toy Computation Graph

In [13]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.name = name
        self.value = init_value
    def evaluate(self):
        return self.value
    def __str__(self):
        return self.name
    
class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() + self.b.evaluate()  # a, b are var/const
    def __str__(self):
        return '{} + {}'.format(self.a, self.b)
    
class Mul(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() * self.b.evaluate()  # a, b are var/const
    def __str__(self):
        return '{} * {}'.format(self.a, self.b)
        

In [14]:
x = Var('x')
y = Var('y')
f = Add(Add(Mul(Mul(x, x), y), y), Const(2))  # f(x,y) = x²y + y + 2

In [15]:
x.value = 3
y.value = 4
f.evaluate()

42

# Computing gradients

In [16]:
from math import sin

def z(x):
    return sin(x**2)

gradients(z, [3])

[-5.46761419430053]

In [17]:
Const.gradient = lambda self, var: Const(0)
Var.gradient = lambda self, var: Const(1) if self is var else Const(0)
Add.gradient = lambda self, var: Add(self.a.gradient(var), self.b.gradient(var))
Mul.gradient = lambda self, var: Add(Mul(self.a.gradient(var), self.b), Mul(self.b.gradient(var), self.a))

x = Var(name='x', init_value=3.)
y = Var(name='y', init_value=4.)
f = Add(Add(Mul(Mul(x, x), y), y), Const(2))  # f(x,y) = x²y + y + 2

dfdx = f.gradient(x)  # 2xy
dfdy = f.gradient(y)  # x² + 1

In [18]:
dfdx.evaluate(), dfdy.evaluate()

(24.0, 10.0)

**Since the output of the gradient() method is fully symbolic, we are not limited to the first order derivatives, we can also compute second order derivatives**, and so on:

In [19]:
d2fdxdx = dfdx.gradient(x)  # 2y
d2fdxdy = dfdx.gradient(y)  # 2x
d2fdydx = dfdy.gradient(x)  # 2x
d2fdydy = dfdy.gradient(y)  # 0

In [20]:
[[d2fdxdx.evaluate(), d2fdxdy.evaluate()],
[d2fdydx.evaluate(), d2fdydy.evaluate()]]

[[8.0, 6.0], [6.0, 0.0]]

## Forward mode autodiff using dual numbers

In [21]:
# python __add__和__radd__: https://blog.csdn.net/u011019726/article/details/77834602

class DualNumber(object):
    def __init__(self, value=0.0, eps=0.0):
        self.value = value
        self.eps = eps
    def __add__(self, b):
        return DualNumber(self.value + self.to_dual(b).value, 
                          self.eps + self.to_dual(b).eps)  # = (value1 + value2) + (eps1 + eps2)
    def __radd__(self,a):
        return self.to_dual(a).__add__(self)
    def __mul__(self, b):
        return DualNumber(self.value * self.to_dual(b).value, 
                          self.value * self.to_dual(b).eps + self.eps * self.to_dual(b).value)
    def __rmul__(self, a):
        return self.to_dual(a).__mul__(self)
    def __str__(self):
        if self.eps:
            return '{:.1f} + {:.1f}ε'.format(self.value, self.eps)
        else:
            return '{:.1f}'.format(self.value)
    def __repr__(self):
        return str(self)    
    # classmethod 修饰符对应的函数不需要实例化，不需要 self 参数，
    # 第一个参数需要是表示自身类的 cls 参数，可以来调用类的属性，类的方法，实例化对象等。
    @classmethod  
    def to_dual(cls, n):
        if hasattr(n, 'value'):
            return n
        else:
            return cls(n)  # 调用类的属性，类的方法
        

In [22]:
3 + DualNumber(3, 4)

6.0 + 4.0ε

In [23]:
DualNumber(3, 4) * DualNumber(5, 7)

15.0 + 41.0ε

In [24]:
x.value = DualNumber(3.0)
y.value = DualNumber(4.0)

f.evaluate()

42.0

In [25]:
x.value = DualNumber(3.0, 1.0)  # 3 + ε
y.value = DualNumber(4.0)  # 4

dfdx = f.evaluate().eps

x.value = DualNumber(3.0)
y.value = DualNumber(4.0, 1.0)

dfdy = f.evaluate().eps

In [26]:
dfdx

24.0

In [27]:
dfdy

10.0

## Reverse mode autodiff

In [28]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        pass
    def __str__(self):
        return str(self.value)
    
class Var(object):
    def __init__(self, name, init_value=0):
        self.name = name
        self.value = init_value
        self.gradient = 0  # initialise the gradient
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        self.gradient += gradient
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    def __str__(self):
        return '{} + {}'.format(self.a, self.b)
    
class Mul(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    def __str__(self):
        return '{} * {}'.format(self.a, self.b)
        

In [29]:
x = Var('x', init_value=3)
y = Var('y', init_value=4)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

result = f.evaluate()
f.backpropagate(1.0)

In [30]:
result

42

In [31]:
x.gradient

24.0

In [32]:
y.gradient

10.0

## Reverse mode autodiff using TensorFlow

In [33]:
import tensorflow as tf

In [34]:
tf.reset_default_graph()

x = tf.Variable(3., name='x')
y = tf.Variable(4., name='y')
f = x*x*y + y + 2

jacobians = tf.gradients(f, [x, y])

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    f_val, jacobians_val = sess.run([f, jacobians])
    
f_val, jacobians_val

Instructions for updating:
Colocations handled automatically by placer.


(42.0, [24.0, 10.0])

In [35]:
hessians_x = tf.gradients(jacobians[0], [x, y])
hessians_y = tf.gradients(jacobians[1], [x, y])

def replace_none_with_zero(tensors):
    return [tensor if tensor is not None else tf.constant(0.) 
            for tensor in tensors]

hessians_x = replace_none_with_zero(hessians_x)
hessians_y = replace_none_with_zero(hessians_y)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    hessians_x_val, hessians_y_val = sess.run([hessians_x, hessians_y])
    
hessians_x_val, hessians_y_val

([8.0, 6.0], [6.0, 0.0])